In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
import os
import optuna

In [10]:
pip freeze

alembic==1.17.1
anyio @ file:///home/conda/feedstock_root/build_artifacts/bld/rattler-build_anyio_1754315087/work
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1733332318622/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1749017159514/work
argon2-cffi-bindings @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi-bindings_1753994790109/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1733584251875/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1733250440834/work
async-lru @ file:///home/conda/feedstock_root/build_artifacts/bld/rattler-build_async-lru_1742153708/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1741918516150/work
babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1738490167835/work
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup4_1756094236201/work
bleach @ file:///home/conda/feedstock_root/build_art

In [7]:
RECEIVALS_PATH = "data/kernel/receivals.csv"
PO_PATH = "data/kernel/purchase_orders.csv"
MAT_PATH = "data/extended/materials.csv"

FORECAST_START = pd.to_datetime("2025-01-01", utc=True)
FORECAST_END = pd.to_datetime("2025-05-31", utc=True)

LAST_RECEIVAL_DATE = pd.Timestamp("2024-12-19", tz="UTC")

def prepareData(receivals_path=RECEIVALS_PATH, po_path=PO_PATH, mat_path=MAT_PATH):
    
    # === Clean receivals ===
    receivals = pd.read_csv(receivals_path)
    receivals = receivals.dropna(subset=["rm_id", "purchase_order_id", "product_id"])
    receivals["rm_id"] = receivals["rm_id"].astype(int)
    receivals["purchase_order_id"] = receivals["purchase_order_id"].astype(int)
    receivals["product_id"] = receivals["product_id"].astype(int)
    receivals["date_arrival"] = pd.to_datetime(receivals["date_arrival"], utc=True)

    # == Clean purchase orders ==
    purchase_orders = pd.read_csv(po_path)
    purchase_orders = purchase_orders[purchase_orders["quantity"] > 0] # Remove negative and 0 quantity orders
    purchase_orders["delivery_date"] = pd.to_datetime(purchase_orders["delivery_date"], utc=True)

    # == Clean materials ==
    materials = pd.read_csv(mat_path)
    materials = materials.dropna(subset=["product_id", "rm_id"])
    materials["product_id"] = materials["product_id"].astype(int)
    materials["rm_id"] = materials["rm_id"].astype(int)

    # Find rm_ids scheduled for 2025
    po_2025 = purchase_orders[purchase_orders["delivery_date"] >= FORECAST_START].copy()
    po_2025 = po_2025.sort_values(by="delivery_date")
    materials_unique = materials.drop_duplicates(subset=["rm_id", "product_id"])

    merged_po2025_mat = po_2025.merge(materials_unique, on=["product_id"], how="left")

    one_year_before = pd.to_datetime("2024-01-01", utc=True) # Only rm_ids that had a delivery in 2024, if not they are likely out of production
    active_rm_ids = receivals.loc[receivals["date_arrival"] >= one_year_before, "rm_id"].unique()

    merged_active = merged_po2025_mat[merged_po2025_mat["rm_id"].isin(active_rm_ids)].copy()
    scheduled_active_rm_ids = merged_active["rm_id"].unique() # 38 found to be active with this method

    # Find scheduled quantity for all active rm_ids/product_ids in 2025.
    qty_rm_id_2025 = merged_active[["rm_id", "product_id", "delivery_date", "quantity"]]
    qty_rm_id_2025["delivery_date"] = qty_rm_id_2025["delivery_date"].dt.date

    # Get daily total receivals per rm_id and mean/std(delay) for each rm_id
    merged = pd.merge(purchase_orders, receivals, on=["purchase_order_id", "purchase_order_item_no"], suffixes=("_receival", "_order"))
    merged = merged.sort_values(by="date_arrival")

    merged["delay"] = (merged["date_arrival"] - merged["delivery_date"]).dt.days
    merged["date_arrival"] = merged["date_arrival"].dt.date
    merged["delivery_date"] = merged["delivery_date"].dt.date

    daily_receivals = merged.groupby(["rm_id", "date_arrival"], as_index=False)["net_weight"].sum()
    daily_receivals["date_arrival"] = pd.to_datetime(daily_receivals["date_arrival"], utc=True)

    delays = merged[["rm_id", "date_arrival", "delay"]]

    delay_stats = merged.groupby("rm_id")["delay"].agg(["mean", "std"]).reset_index()

    purchase_orders["delivery_date"] = purchase_orders["delivery_date"].dt.date
    receivals["date_arrival"] = receivals["date_arrival"].dt.date
    daily_receivals["date_arrival"] = daily_receivals["date_arrival"].dt.date 

    return receivals, purchase_orders, daily_receivals, qty_rm_id_2025, scheduled_active_rm_ids, delays, delay_stats, active_rm_ids

def getExpectedQty(rm_id, purchase_orders, mapping, forecast_start, forecast_end):
    prod_id = getProdId(rm_id, mapping)
    if prod_id is None:
        return 0.0
    expected_qty_product_id = purchase_orders.loc[
        (purchase_orders["product_id"] == prod_id) &
        (purchase_orders["delivery_date"] >= forecast_start) &
        (purchase_orders["delivery_date"] <= forecast_end),
        "quantity"
    ].sum()

    return expected_qty_product_id

def getProdId(rm_id, mapping):
    row = mapping.loc[mapping["rm_id"] == rm_id]
    if row.empty:
        return None
    return row["product_id"].iloc[0]

def prob_rm_id(rm_id, mapping, receivals, forecast_start, days):
    # Find prob of
    prod_id = getProdId(rm_id, mapping)
    check_from = (forecast_start - pd.Timedelta(days=days))
    recent_prodId = receivals.loc[
        (receivals["product_id"] == prod_id) & # can be a problem with float vs int here. Maybe drop NaNs in receivals and initfy
        (receivals["date_arrival"] >= check_from) &
        (receivals["date_arrival"] < forecast_start)
    ]
    
    if recent_prodId.empty:
        return 1.0
     
    total_count = len(recent_prodId)
    rm_count = (recent_prodId["rm_id"] == rm_id).sum()
    return rm_count/total_count

def get_exp_qty_rm_id(rm_id, mapping, receivals, purchase_orders, forecast_start, forecast_end, histDays):
    totExpectedQty = getExpectedQty(rm_id, purchase_orders, mapping, forecast_start, forecast_end)
    probRmID = prob_rm_id(rm_id, mapping, receivals, forecast_start, histDays)
    expectQty = totExpectedQty*probRmID
    bufferQty = totExpectedQty-expectQty
    return expectQty, bufferQty

def build_features(rm_id, forecast_start, forecast_end, daily_receivals, purchase_orders, delays, mapping, receivals):
    hist_delays = delays[delays["date_arrival"] < forecast_start]
    hist_daily_receivals = daily_receivals[daily_receivals["date_arrival"] < forecast_start]
    
    recent_from_150 = (forecast_start - pd.Timedelta(days=150))
    recent_from_60 = (forecast_start - pd.Timedelta(days=60))
    recent_from_30 = (forecast_start - pd.Timedelta(days=30))
    
    recent_150 = daily_receivals[(daily_receivals["date_arrival"] >= recent_from_150) 
                           & (daily_receivals["date_arrival"] < forecast_start)].copy()
    
    recent_60 = daily_receivals[(daily_receivals["date_arrival"] >= recent_from_60) 
                           & (daily_receivals["date_arrival"] < forecast_start)].copy()
    
    recent_30 = daily_receivals[(daily_receivals["date_arrival"] >= recent_from_30) 
                           & (daily_receivals["date_arrival"] < forecast_start)].copy()
    

    full_date_range_150 = pd.date_range(
        start=recent_from_150, 
        end=(forecast_start - pd.Timedelta(days=1)), 
        freq="D"
    )

    full_date_range_60 = pd.date_range(
        start=recent_from_60, 
        end=(forecast_start - pd.Timedelta(days=1)), 
        freq="D"
    )

    full_date_range_30 = pd.date_range(
        start=recent_from_30, 
        end=(forecast_start - pd.Timedelta(days=1)), 
        freq="D"
    )

    recent_150_full = (
        recent_150
        .set_index("date_arrival")
        .reindex(full_date_range_150, fill_value=0)
        .rename_axis("date_arrival")
        .reset_index()
    )

    recent_60_full = (
        recent_60
        .set_index("date_arrival")
        .reindex(full_date_range_60, fill_value=0)
        .rename_axis("date_arrival")
        .reset_index()
    )

    recent_30_full = (
        recent_30
        .set_index("date_arrival")
        .reindex(full_date_range_30, fill_value=0)
        .rename_axis("date_arrival")
        .reset_index()
    )

    recent_150_full["rm_id"] = rm_id
    recent_60_full["rm_id"] = rm_id
    recent_30_full["rm_id"] = rm_id

    total_expected_quantity = purchase_orders.loc[
        (purchase_orders["delivery_date"] >= forecast_start) &
        (purchase_orders["delivery_date"] <= forecast_end),
        "quantity"
    ].sum()

    mean_daily_weight_150 = recent_150_full["net_weight"].mean()
    mean_daily_weight_60 = recent_60_full["net_weight"].mean()
    mean_daily_weight_30 = recent_30_full["net_weight"].mean()
    window_length = (forecast_end-forecast_start).days + 1

    expectQty_rm, bufferQty_rm = get_exp_qty_rm_id(rm_id, mapping, receivals, purchase_orders, forecast_start, forecast_end, 150)

    last_delivery = hist_daily_receivals["date_arrival"].max()
    if pd.isna(last_delivery):
        days_since_last_delivery = np.nan
    else:
        days_since_last_delivery = (forecast_start - last_delivery).days

    features = {
        "rm_id": rm_id,
        "forecast_start": forecast_start,
        "forecast_end": forecast_end,
        "window_length": window_length,
        "mean_daily_weight_150": mean_daily_weight_150,
        "mean_daily_weight_60": mean_daily_weight_60,
        "mean_daily_weight_30": mean_daily_weight_30,
        "days_since_last_delivery": days_since_last_delivery,
        "num_deliveries_last_150": len(recent_150),
        "avg_delivery_time": hist_delays["delay"].mean(),
        "std_delivery_time": hist_delays["delay"].std(),
        "total_expected_qty": total_expected_quantity,
        "expected_qty_rm_id": expectQty_rm,
        "buffer_qty": bufferQty_rm

    }

    # Can make month and day etc on forecast_end

    return features

def buildTrainSet(receivals, daily_receivals, purchase_orders, delays, active_rm_ids, mapping, output_path):
    train_rows = []
    window_lengths = range(1, 151) 
    start_dates = pd.date_range(pd.to_datetime("2018-01-01", utc=True), FORECAST_START, freq="14D")  # slide every 14 days
    
    receivals_by_rm = {rid: df for rid, df in receivals.groupby("rm_id")}
    daily_by_rm = {rid: df for rid, df in daily_receivals.groupby("rm_id")}
    delays_by_rm = {rid: df for rid, df in delays.groupby("rm_id")}
    
    for rm_id in tqdm(active_rm_ids):
        print("Processing rm_id: ", rm_id)

        for wl in tqdm(window_lengths):
            for start in start_dates:
                end = start + pd.Timedelta(days=wl - 1)
                if end >= LAST_RECEIVAL_DATE: # last date in receivals
                    break
                feats = build_features(rm_id, start.date(), end.date(), 
                                       daily_by_rm.get(rm_id, pd.DataFrame()), 
                                       purchase_orders, 
                                       delays_by_rm.get(rm_id, pd.DataFrame()), 
                                       mapping, receivals_by_rm.get(rm_id, pd.DataFrame()))

                cum_weight = daily_receivals.loc[
                    (daily_receivals["rm_id"] == rm_id) &
                    (daily_receivals["date_arrival"] >= start.date()) &
                    (daily_receivals["date_arrival"] <= end.date()),
                    "net_weight"
                ].sum()

                feats["cum_weight"] = cum_weight
                train_rows.append(feats)

    df_train = pd.DataFrame(train_rows)

    df_train.to_csv(output_path, index=False)
    
    return df_train


In [4]:
def quantile_loss(y_true, y_pred, alpha=0.2):
    err = y_true - y_pred
    return np.mean(np.maximum(alpha * err, (alpha - 1) * err))

def objective(trial, X_train, y_train, X_valid, y_valid):
    params = dict(
        objective="quantile",
        alpha=0.2,
        learning_rate=trial.suggest_float("learning_rate", 0.001, 0.05, log=True),
        num_leaves=trial.suggest_int("num_leaves", 15, 127, log=True),
        max_depth=trial.suggest_int("max_depth", 3, 9),
        subsample=trial.suggest_float("subsample", 0.6, 1.0),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.6, 1.0),
        reg_alpha=trial.suggest_float("reg_alpha", 0.0, 1.0),
        reg_lambda=trial.suggest_float("reg_lambda", 0.0, 1.0),
        min_child_samples=trial.suggest_int("min_child_samples", 10, 50),
        random_state=42,
        n_jobs=-1
    )

    model = lgb.LGBMRegressor(**params, n_estimators=5000)

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="quantile",
        categorical_feature=["rm_id"],
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )

    preds = model.predict(X_valid)
    return quantile_loss(y_valid, preds, alpha=0.2)


def train_global_model_lgbm_optuna(df_train, model_dir):

    # Define training/validation split by time
    cutoff = LAST_RECEIVAL_DATE - pd.Timedelta(days=150)
    df_train["forecast_start"] = pd.to_datetime(df_train["forecast_start"], utc=True)
    df_train["forecast_end"] = pd.to_datetime(df_train["forecast_end"], utc=True)

    train = df_train[df_train["forecast_end"] < cutoff]
    valid = df_train[df_train["forecast_start"] >= cutoff]

    X_train = train.drop(columns=["cum_weight", "forecast_start", "forecast_end"])
    y_train = train["cum_weight"]
    X_valid = valid.drop(columns=["cum_weight", "forecast_start", "forecast_end"])
    y_valid = valid["cum_weight"]

    # === Optuna tuning ===
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_valid, y_valid), n_trials=10)
    best_params = study.best_params
    print("Best params:", best_params)

    # Add fixed parameters
    best_params.update({
        "objective": "quantile",
        "alpha": 0.2,
        "random_state": 42,
        "n_jobs": -1,
        "verbosity": -1
    })

    # === Retrain model with best params ===
    best_model = lgb.LGBMRegressor(**best_params, n_estimators=1000)
    best_model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="quantile",
        categorical_feature=["rm_id"],
        callbacks=[lgb.early_stopping(200)]
    )

    best_iteration = best_model.best_iteration_
    print(f"Best iteration found: {best_iteration}")

    # === Train final model on all data ===
    X_full = df_train.drop(columns=["cum_weight", "forecast_start", "forecast_end"])
    y_full = df_train["cum_weight"]

    final_model = lgb.LGBMRegressor(**best_params, n_estimators=best_iteration)
    final_model.fit(X_full, y_full, categorical_feature=["rm_id"])

    final_model.booster_.save_model(model_dir)
    return final_model


In [16]:
def forecast2025(model, forecast_start, forecast_end, active_rm_ids, daily_receivals, purchase_orders, delays,
             mapping, receivals, pred_path):

    print("Forecasting for 2025!")
    pred_rows = []

    forecast_range = pd.date_range(forecast_start, forecast_end - pd.Timedelta(days=1))

    receivals_by_rm = {rid: df for rid, df in receivals.groupby("rm_id")}
    daily_by_rm = {rid: df for rid, df in daily_receivals.groupby("rm_id")}
    delays_by_rm = {rid: df for rid, df in delays.groupby("rm_id")}
    
    for rm_id in tqdm(active_rm_ids):
        print("Building features for rm_id ", rm_id)
        # Build features for the given rm_id and forecast window
        for date in forecast_range:
            end_date = date.date() + pd.Timedelta(days=1)
            feats = build_features(rm_id, forecast_start.date(), end_date,
                                                    daily_by_rm.get(rm_id, pd.DataFrame()), 
                                                    purchase_orders, 
                                                    delays_by_rm.get(rm_id, pd.DataFrame()), 
                                                    mapping, 
                                                    receivals_by_rm.get(rm_id, pd.DataFrame()))
            X_test = pd.DataFrame([feats]).drop(columns=["forecast_start", "forecast_end"])

            # Predict using trained model
            pred_cum_weight = model.predict(X_test)[0]
            pred_cum_weight = max(pred_cum_weight, 0)

            pred_rows.append({
                "rm_id": rm_id,
                "forecast_start_date": forecast_start.date(),
                "forecast_end_date": end_date,
                "cum_weight": pred_cum_weight
            })


    pred_df = pd.DataFrame(pred_rows)

    pred_df.to_csv(pred_path)
        
    return pred_df

In [2]:
def createSubmission(pred_path, pred_map_path, sub_path):
    forecast = pd.read_csv(pred_path)
    mapping = pd.read_csv(pred_map_path)

    merged = mapping.merge(
        forecast,
        on=["rm_id", "forecast_start_date", "forecast_end_date"],
        how="left"
    )

    # Replace missing cum_weight values with 0
    merged["cum_weight"] = merged["cum_weight"].fillna(0)

    # Prepare submission format
    submission = merged[["ID", "cum_weight"]].rename(columns={"cum_weight": "predicted_weight"})

    # Save to CSV
    submission.to_csv(sub_path, index=False)
    return 

In [9]:
def main():
    # == Prepare and build features and training set ==
    receivals, purchase_orders, daily_receivals, qty_2025_rm_id, scheduled_rm_ids, delays, delay_stats, active_rm_ids = prepareData()
    df_train = buildTrainSet(receivals, daily_receivals, purchase_orders, delays, scheduled_rm_ids, qty_2025_rm_id, "trainingSet.csv")
    #df_train = pd.read_csv("trainingSet.csv")

    # == Train global LGBM ==
    model_dir = "lgbm_global_optuna.txt"
    model = train_global_model_lgbm_optuna(df_train, model_dir=model_dir)
    booster = model.booster_

    # == Forecast for 2025 ==
    prediction_path = "lgbm_global.csv"
    
    forecast2025(booster, FORECAST_START, FORECAST_END, scheduled_rm_ids, 
                                 daily_receivals, purchase_orders, delays, qty_2025_rm_id, receivals, 
                                 pred_path=prediction_path)
    
    # == Create submission == 
    createSubmission(prediction_path, "data/prediction_mapping.csv", "submission2.csv")
    return

if __name__ == "__main__":
    main()

/var/folders/1y/z5mwdhtx1vq9zh65ghs4txg40000gn/T/ipykernel_4684/2652312477.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qty_rm_id_2025["delivery_date"] = qty_rm_id_2025["delivery_date"].dt.date
  0%|          | 0/38 [00:00<?, ?it/s]

Processing rm_id:  2161


  0%|          | 0/38 [00:15<?, ?it/s]


KeyboardInterrupt: 